In [57]:
import pandas as pd
import ijson

import ast
import os

In [18]:
filename = "nde_hub.nde_all_prod_20240214_b9kgi6tc.json"
old_data = []

with open(filename, 'r') as file:
    objects = ijson.items(file, 'item')
    for obj in objects:
        old_data.append(obj)

In [19]:
old_data = pd.DataFrame(old_data)

In [20]:
old_data.columns

Index(['_id', 'date', 'description', 'includedInDataCatalog', 'name', 'url',
       'abstract'],
      dtype='object')

In [21]:
filename = "nde_hub.nde_all_prod_20240531_lcmzlshe.json"
new_data = []

with open(filename, 'r') as file:
    objects = ijson.items(file, 'item')
    for obj in objects:
        new_data.append(obj)

In [22]:
new_data = pd.DataFrame(new_data)

In [25]:
def convert_dict_to_list(x):
    if isinstance(x, list):
        return x[0]['name']
    else:
        return x['name']


new_data['includedInDataCatalog'] = new_data['includedInDataCatalog'].apply(convert_dict_to_list)
# old_data['includedInDataCatalog'] = old_data['includedInDataCatalog'].apply(convert_dict_to_list)

TypeError: string indices must be integers

In [33]:
old_data[old_data['includedInDataCatalog'] == 'Omics Discovery Index (OmicsDI)']['_id'].to_csv('old_omics.csv', index=False)

In [34]:
new_data[new_data['includedInDataCatalog'] == 'Omics Discovery Index (OmicsDI)']['_id'].to_csv('new_omics.csv', index=False)

In [39]:
old_ids = old_data[old_data['includedInDataCatalog'] == 'Omics Discovery Index (OmicsDI)']['_id']
new_ids = new_data[new_data['includedInDataCatalog'] == 'Omics Discovery Index (OmicsDI)']['_id']

In [44]:
old_ids = old_ids.str.replace('.', '-').str.lower()

In [55]:
print(len(old_ids), len(new_ids))

1802736 2073853


In [50]:
num_ids_in_old_data = old_ids.isin(new_ids).sum()
print(num_ids_in_old_data)

1208906


In [53]:
new_ids_not_in_old = new_ids[~new_ids.isin(old_ids)]
print(len(new_ids_not_in_old))


864947


In [60]:
filename = "nde_hub_src.hca.json"
repo = []

with open(filename, 'r') as file:
    objects = ijson.items(file, 'item')
    for obj in objects:
        repo.append(obj)

In [62]:
repo = pd.DataFrame(repo)

In [64]:
repo['includedInDataCatalog'] = repo['includedInDataCatalog'].apply(convert_dict_to_list)

In [66]:
repo.to_csv('humanatlas.csv', index=False)

## Fix IDs

Lowercase all IDs

Dryad, OmicsDI, MicrobiomeDB, Dockstore

In [19]:
## DRYAD

regex_pattern = r"doi\.org:\d+\.\d+:dryad\.?|doi\.org:\d+\.\d+:"
old_data['_id'] = old_data['_id'].str.replace(regex_pattern, '', regex=True)

In [20]:
old_data[old_data['includedInDataCatalog'] == 'Dryad Digital Repository']['_id']

89256                                  DRYAD_5xcp-0d46
89257                                     DRYAD_R30029
89258                                     DRYAD_R3059P
89259                                     DRYAD_R30C70
89260                                     DRYAD_R30D46
                              ...                     
143459                                    DRYAD_D1V63M
143460                                    DRYAD_D1WK8M
143461                                    DRYAD_D1XC97
143462                                    DRYAD_D1ZS7X
143463    DRYAD_doi.org:badcat--10.5068:D1V665--badcat
Name: _id, Length: 54208, dtype: object

In [59]:
old_data['_id'] = old_data['_id'].str.lower()
new_data['_id'] = new_data['_id'].str.lower()

## Merge and filter new records

In [60]:
# merged_data = new_data.merge(old_data, on='name', indicator=True, how='left')

merged_data = new_data.merge(old_data, indicator=True, how='left')
rows_not_in_old_data = merged_data[merged_data['_merge'] == 'left_only']

In [61]:
rows_not_in_old_data['includedInDataCatalog'].unique()

array(['MalariaGEN', 'VDJServer', 'Mendeley', 'ImmPort', 'ReframeDB',
       'heiDATA', 'UNC Dataverse', 'Recherche Data Gouv', 'DataverseNO',
       'Texas Data Repository', 'ICRISAT Dataverse', 'DataSuds',
       'DataverseNL', 'Qualitative Data Repository', 'Borealis',
       'Harvard Dataverse', 'Abacus Data Network',
       'World Agroforestry (ICRAF)',
       'World Agroforestry - Research Data Repository',
       'CIMMYT Research Data & Software Repository Network',
       'Pontificia Universidad Católica del Perú ',
       'AccessClinicalData@NIAID', 'Dryad Digital Repository',
       'MicrobiomeDB', 'Omics Discovery Index (OmicsDI)', 'NCBI GEO'],
      dtype=object)

In [62]:
# Can't filter 'Omics Discovery Index (OmicsDI)' properly, so ignoring
repos = ['MalariaGEN', 'VDJServer', 'Mendeley', 'Immport', 'ReframeDB', 'Harvard Dataverse', 'AccessClinicalData@NIAID', 'Dryad Digital Repository', 'NCBI GEO', 'MicrobiomeDB']  

filtered_data = rows_not_in_old_data[rows_not_in_old_data['includedInDataCatalog'].isin(repos)]

In [63]:
filtered_data['includedInDataCatalog'].unique()

array(['MalariaGEN', 'VDJServer', 'Mendeley', 'ReframeDB',
       'Harvard Dataverse', 'AccessClinicalData@NIAID',
       'Dryad Digital Repository', 'MicrobiomeDB', 'NCBI GEO'],
      dtype=object)

In [64]:
filtered_data.shape

(94370, 8)

In [65]:
filtered_data.reset_index(drop=True, inplace=True)

In [67]:
filtered_data.to_csv('incoming_data.csv', index=False)